In [80]:
import sys
import json
default = {'training_folder': '../../'} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../'}


In [81]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
import numpy as np
from typing import Literal

on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
io_engine = 'pyogrio' if on_lambda else 'pyogrio' #or fiona

In [82]:
base_folder = argv['training_folder']

In [83]:
name = 'Dallas_CT_jp' ###
scenario = 'dallas'

zones = gpd.read_file(os.path.join(base_folder,'inputs/zones', name+'.geojson'), engine=io_engine)  


In [84]:
zones

,OBJECTID,NAME,ALAND,AWATER,pop_total,pop_5to19,pop_20to54,pop_55up,pop_dens,pop_urban,pop_rural,pop_u_and_r_undefined,pct_urban,pct_rural,emp_total,emp_dens,geometry
0,1,Census Tract 301.01,119235977,1249939,3595,848,1538,1209,30.2,0,3595,0,0.0,100.0,391,3.279212,"MULTIPOLYGON (((-10728230.405 3923606.785, -10..."
1,2,Census Tract 301.02,200441944,2109723,4282,883,1758,1641,21.4,0,4282,0,0.0,100.0,250,1.247244,"MULTIPOLYGON (((-10729485.645 3948229.330, -10..."
2,3,Census Tract 302.01,91250954,1329423,6175,1373,2821,1981,67.7,2929,3246,0,47.4,52.6,1118,12.251927,"MULTIPOLYGON (((-10739519.095 3928153.909, -10..."
3,4,Census Tract 302.02,151377249,2417266,6345,1267,2868,2210,41.9,0,6345,0,0.0,100.0,504,3.329430,"MULTIPOLYGON (((-10759455.197 3948689.220, -10..."
4,5,Census Tract 302.04,15792805,2872,7554,1980,3735,1839,478.3,6524,1030,0,86.4,13.6,649,41.094663,"MULTIPOLYGON (((-10749559.783 3948510.409, -10..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,1700,Census Tract 1506.03,58516666,1098332,4990,1147,2190,1653,85.3,29,4961,0,0.6,99.4,750,12.816861,"MULTIPOLYGON (((-10851312.277 3902134.520, -10..."
1700,1701,Census Tract 1506.04,105304862,210841,4590,918,1914,1758,43.6,0,4590,0,0.0,100.0,84,0.797684,"MULTIPOLYGON (((-10858486.931 3894534.102, -10..."
1701,1702,Census Tract 1506.05,96279862,1217301,4901,967,2056,1878,50.9,0,4901,0,0.0,100.0,1127,11.705459,"MULTIPOLYGON (((-10858847.162 3908530.711, -10..."
1702,1703,Census Tract 1506.06,49021523,0,3283,772,1531,980,67.0,0,3283,0,0.0,100.0,963,19.644433,"MULTIPOLYGON (((-10841989.157 3908356.369, -10..."


In [85]:
zones = zones.set_index('OBJECTID')
zones.index.name = 'index'

In [86]:
zones = zones.to_crs(4326)

In [87]:
pd.set_option('display.max_column', None)
zones.head()

,NAME,ALAND,AWATER,pop_total,pop_5to19,pop_20to54,pop_55up,pop_dens,pop_urban,pop_rural,pop_u_and_r_undefined,pct_urban,pct_rural,emp_total,emp_dens,geometry
index,,,,,,,,,,,,,,,,
1,Census Tract 301.01,119235977,1249939,3595,848,1538,1209,30.2,0,3595,0,0.0,100.0,391,3.279212,"MULTIPOLYGON (((-96.37333 33.21297, -96.37352 ..."
2,Census Tract 301.02,200441944,2109723,4282,883,1758,1641,21.4,0,4282,0,0.0,100.0,250,1.247244,"MULTIPOLYGON (((-96.38461 33.39783, -96.38485 ..."
3,Census Tract 302.01,91250954,1329423,6175,1373,2821,1981,67.7,2929,3246,0,47.4,52.6,1118,12.251927,"MULTIPOLYGON (((-96.47474 33.24714, -96.47433 ..."
4,Census Tract 302.02,151377249,2417266,6345,1267,2868,2210,41.9,0,6345,0,0.0,100.0,504,3.329430,"MULTIPOLYGON (((-96.65383 33.40128, -96.65649 ..."
5,Census Tract 302.04,15792805,2872,7554,1980,3735,1839,478.3,6524,1030,0,86.4,13.6,649,41.094663,"MULTIPOLYGON (((-96.56494 33.39994, -96.56498 ..."


In [88]:
tmp_zones = zones[['pop_dens', 'pop_total','emp_total', 'geometry', 'NAME']].rename(columns={'pop_dens': 'density','pop_total':'population', 'emp_total': 'jobs' })

In [89]:
tmp_zones

,density,population,jobs,geometry,NAME
index,,,,,
1,30.2,3595,391,"MULTIPOLYGON (((-96.37333 33.21297, -96.37352 ...",Census Tract 301.01
2,21.4,4282,250,"MULTIPOLYGON (((-96.38461 33.39783, -96.38485 ...",Census Tract 301.02
3,67.7,6175,1118,"MULTIPOLYGON (((-96.47474 33.24714, -96.47433 ...",Census Tract 302.01
4,41.9,6345,504,"MULTIPOLYGON (((-96.65383 33.40128, -96.65649 ...",Census Tract 302.02
5,478.3,7554,649,"MULTIPOLYGON (((-96.56494 33.39994, -96.56498 ...",Census Tract 302.04
...,...,...,...,...,...
1700,85.3,4990,750,"MULTIPOLYGON (((-97.47900 33.05145, -97.47911 ...",Census Tract 1506.03
1701,43.6,4590,84,"MULTIPOLYGON (((-97.54345 32.99420, -97.54344 ...",Census Tract 1506.04
1702,50.9,4901,1127,"MULTIPOLYGON (((-97.54668 33.09959, -97.54688 ...",Census Tract 1506.05


In [90]:
output_folder= base_folder +'scenarios/{scen}/inputs/'.format(scen = scenario)
tmp_zones.to_file(output_folder + 'zones.geojson',driver='GeoJSON')

In [91]:
# find meters CRS
from shapely.geometry import Point, LineString
# from quetzal_cyclops
def get_epsg(lat: float, lon: float) -> int:
    '''
    lat, lon or y, x
    return EPSG in meter for a given (lat,lon)
    lat is north south 
    lon is est west
    '''
    return int(32700 - round((45 + lat) / 90, 0) * 100 + round((183 + lon) / 6, 0))

centroid = [*LineString(zones.centroid.values).centroid.coords][0]
crs = get_epsg(centroid[1],centroid[0])
crs

/tmp/ipykernel_15883/2198738084.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = [*LineString(zones.centroid.values).centroid.coords][0]


32614

In [92]:
zones['area (km2)'] = zones.to_crs(crs).area
zones['area (km2)'].sum()

22863675104.156017